# Running Tune experiments with BayesOpt

In this tutorial we introduce BayesOpt, while running a simple Ray Tune experiment. Tune’s Search Algorithms integrate with BayesOpt and, as a result, allow you to seamlessly scale up a BayesOpt optimization process - without sacrificing performance.

BayesOpt is a constrained global optimization package utilizing Bayesian inference on gaussian processes, where the emphasis is on finding the maximum value of an unknown function in as few iterations as possible. BayesOpt's techniques are particularly suited for optimization of high cost functions, situations where the balance between exploration and exploitation is important. Therefore BayesOpt falls in the domain of "derivative-free" and "black-box" optimization. In this example we minimize a simple objective to briefly demonstrate the usage of BayesOpt with Ray Tune via `BayesOptSearch`, including conditional search spaces. It's useful to keep in mind that despite the emphasis on machine learning experiments, Ray Tune optimizes any implicit or explicit objective. Here we assume `bayesian-optimization==1.2.0` library is installed. To learn more, please refer to [BayesOpt website](https://github.com/fmfn/BayesianOptimization).

In [1]:
# !pip install ray[tune]
!pip install bayesian-optimization==1.2.0

Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

In [2]:
import time

import ray
from ray import tune
from ray.air import session
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.bayesopt import BayesOptSearch

Let's start by defining a simple evaluation function.
We artificially sleep for a bit (`0.1` seconds) to simulate a long-running ML experiment.
This setup assumes that we're running multiple `step`s of an experiment and try to tune two hyperparameters,
namely `width` and `height`.

In [3]:
def evaluate(step, width, height):
    time.sleep(0.1)
    return (0.1 + width * step / 100) ** (-1) + height * 0.1

Next, our ``objective`` function takes a Tune ``config``, evaluates the `score` of your experiment in a training loop,
and uses `session.report` to report the `score` back to Tune.

In [4]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"])
        session.report({"iterations": step, "mean_loss": score})

In [5]:
ray.init(configure_logging=False)

Python version:,3.7.7
Ray version:,3.0.0.dev0
Dashboard:,http://127.0.0.1:8266


Now we define the search algorithm built from `BayesOptSearch`, constrained  to a maximum of `4` concurrent trials with a `ConcurrencyLimiter`.

In [6]:
algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0})
algo = ConcurrencyLimiter(algo, max_concurrent=4)

The number of samples is the number of hyperparameter combinations that will be tried out. This Tune run is set to `1000` samples.
(you can decrease this if it takes too long on your machine).

In [7]:
num_samples = 1000

In [8]:
# If 1000 samples take too long, you can reduce this number.
# We override this number here for our smoke tests.
num_samples = 10

Next we define a search space. The critical assumption is that the optimal hyperparameters live within this space. Yet, if the space is very large, then those hyperparameters may be difficult to find in a short amount of time.

In [9]:
search_space = {
    "steps": 100,
    "width": tune.uniform(0, 20),
    "height": tune.uniform(-100, 100),
}

Finally, we run the experiment to `"min"`imize the "mean_loss" of the `objective` by searching `search_config` via `algo`, `num_samples` times. This previous sentence is fully characterizes the search problem we aim to solve. With this in mind, notice how efficient it is to execute `tuner.fit()`.

In [10]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        num_samples=num_samples,
    ),
    param_space=search_space,
)
results = tuner.fit()

Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,height,width,loss,iter,total time (s),iterations,neg_mean_loss
objective_c9daa5d4,TERMINATED,127.0.0.1:46960,-25.092,19.0143,-2.45636,100,10.9865,99,2.45636
objective_cb9bc830,TERMINATED,127.0.0.1:46968,46.3988,11.9732,4.72354,100,11.5661,99,-4.72354
objective_cb9d338c,TERMINATED,127.0.0.1:46969,-68.7963,3.11989,-6.56602,100,11.648,99,6.56602
objective_cb9e97e0,TERMINATED,127.0.0.1:46970,-88.3833,17.3235,-8.78036,100,11.6948,99,8.78036
objective_d229961e,TERMINATED,127.0.0.1:47009,20.223,14.1615,2.09312,100,10.8549,99,-2.09312
objective_d42ac71c,TERMINATED,127.0.0.1:47036,-95.8831,19.3982,-9.53651,100,10.7931,99,9.53651
objective_d43ca61c,TERMINATED,127.0.0.1:47039,66.4885,4.24678,6.88118,100,10.7606,99,-6.88118
objective_d43fb190,TERMINATED,127.0.0.1:47040,-63.635,3.66809,-6.09551,100,10.7997,99,6.09551
objective_da1ff46c,TERMINATED,127.0.0.1:47057,-39.1516,10.4951,-3.81983,100,10.7762,99,3.81983
objective_dc25c796,TERMINATED,127.0.0.1:47062,-13.611,5.82458,-1.19064,100,10.7213,99,1.19064


Result for objective_c9daa5d4:
  date: 2022-07-22_15-30-12
  done: false
  experiment_id: 422a6d2a512a470480e33913d7825a7a
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 7.490802376947249
  neg_mean_loss: -7.490802376947249
  node_ip: 127.0.0.1
  pid: 46960
  time_since_restore: 0.1042318344116211
  time_this_iter_s: 0.1042318344116211
  time_total_s: 0.1042318344116211
  timestamp: 1658500212
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: c9daa5d4
  warmup_time: 0.0032601356506347656
  
Result for objective_cb9bc830:
  date: 2022-07-22_15-30-15
  done: false
  experiment_id: 3a9a6bef89ec4b57bd0fa24dd3b407e6
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 14.639878836228101
  neg_mean_loss: -14.639878836228101
  node_ip: 127.0.0.1
  pid: 46968
  time_since_restore: 0.10442280769348145
  time_this_iter_s: 0.10442280769348145
  time_total_s: 0.10442280769348145
  timestamp: 1658

Result for objective_d42ac71c:
  date: 2022-07-22_15-30-29
  done: false
  experiment_id: 3fdfaecb7adc4c5cb54c0aa76849d532
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 0.41168988591604894
  neg_mean_loss: -0.41168988591604894
  node_ip: 127.0.0.1
  pid: 47036
  time_since_restore: 0.10324597358703613
  time_this_iter_s: 0.10324597358703613
  time_total_s: 0.10324597358703613
  timestamp: 1658500229
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d42ac71c
  warmup_time: 0.0028409957885742188
  
Result for objective_d43ca61c:
  date: 2022-07-22_15-30-29
  done: false
  experiment_id: 8f92f519ea5443be9efd6f4a8937b8ee
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 16.648852816008436
  neg_mean_loss: -16.648852816008436
  node_ip: 127.0.0.1
  pid: 47039
  time_since_restore: 0.10412001609802246
  time_this_iter_s: 0.10412001609802246
  time_total_s: 0.10412001609802246
  timestam

Result for objective_dc25c796:
  date: 2022-07-22_15-30-42
  done: false
  experiment_id: c0f302c32b284f8e99dbdfa90657ee7d
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 8.638900372842315
  neg_mean_loss: -8.638900372842315
  node_ip: 127.0.0.1
  pid: 47062
  time_since_restore: 0.10459494590759277
  time_this_iter_s: 0.10459494590759277
  time_total_s: 0.10459494590759277
  timestamp: 1658500242
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: dc25c796
  warmup_time: 0.002794981002807617
  
Result for objective_da1ff46c:
  date: 2022-07-22_15-30-44
  done: false
  experiment_id: 9163132451a14ace8ddf394aeaae9018
  hostname: Kais-MacBook-Pro.local
  iterations: 47
  iterations_since_restore: 48
  mean_loss: -3.7164550549457847
  neg_mean_loss: 3.7164550549457847
  node_ip: 127.0.0.1
  pid: 47057
  time_since_restore: 5.180424928665161
  time_this_iter_s: 0.10843396186828613
  time_total_s: 5.180424928665161
  timestamp: 1658

Here are the hyperparamters found to minimize the mean loss of the defined objective.

In [11]:
print("Best hyperparameters found were: ", results.get_best_result().config)

Best hyperparameters found were:  {'steps': 100, 'width': 19.398197043239886, 'height': -95.88310114083951}


In [12]:
ray.shutdown()